In [1]:
import pandas as pd
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np  
import os
import glob

In [46]:
# Função para listar todos os arquivos CSV em um diretório
def listar_arquivos_csv(diretorio):
    return [f for f in os.listdir(diretorio) if f.endswith(".csv")]

df_citys = pd.read_csv('../data_sus/muni_lat_long.csv', index_col=0)

def invert_lines_columns(diretorio):
    # Listando todos os arquivos CSV no diretório
    arquivos_csv = listar_arquivos_csv(diretorio)
    columns = ['data_week','casos_est','casos_est_min','casos_est_max','casos','proba_disse>1',
               'incidência_100khab','disseminação','população','tempmin','umidmax',
               'umidmed','umidmin','tempmed','tempmax','municipio']
    for arquivo in arquivos_csv:
        # Caminho completo do arquivo
        caminho_arquivo = os.path.join(diretorio, arquivo)
        
        # Lendo o arquivo CSV em um DataFrame sem cabeçalhos
        df = pd.read_csv(caminho_arquivo)
        
        # Transpor o DataFrame e ajustar os nomes das colunas
        df = df.T
        df.columns = df.iloc[0]  # Define a primeira linha como nomes das colunas
        df = df[1:].reset_index(drop=True)  # Remove a linha que virou cabeçalho

        # Adicionar o nome do município como uma nova coluna
        nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0].replace("_", " ")
        df['municipio'] = nome_arquivo_sem_extensao.title()

        # Renomear as colunas
        df.rename(columns={'data_iniSE': 'data_week', 'p_rt1': 'proba_disse>1', 
                           'p_inc100k': 'incidência_100khab', 'Rt': 'disseminação', 
                           'pop': 'população'}, inplace=True)

        # Selecionar e ordenar as colunas
        df = df[columns]

        # Ordenar por data_week
        df = df.sort_values(['data_week'])

        # Converter colunas para numérico e preencher valores ausentes
        for col in ['tempmin', 'umidmax', 'umidmed', 'umidmin', 'tempmed', 'tempmax']:
            if col in df.columns:
                # Converter valores para numérico (ignorar erros caso haja valores inválidos)
                df[col] = pd.to_numeric(df[col], errors='coerce')
                # Preencher valores ausentes com a mediana
                df[col].fillna(df[col].median(), inplace=True)

        # Salvar o DataFrame modificado no arquivo CSV
        df.to_csv(caminho_arquivo, index=False)
        print(f"Arquivo {arquivo} modificado e salvo com novas colunas.")
# Especificando o diretório onde estão os arquivos CSV
diretorio = "../data_sus/new_daata"

# Chamando a função para processar os arquivos CSV
invert_lines_columns(diretorio)



KeyError: "['data_week', 'casos_est', 'casos_est_min', 'casos_est_max', 'casos', 'proba_disse>1', 'incidência_100khab', 'disseminação', 'população', 'tempmin', 'umidmax', 'umidmed', 'umidmin', 'tempmed', 'tempmax'] not in index"

In [54]:
df = pd.concat([pd.read_csv(f) for f in glob.glob('../data_sus/new_data/*.csv')], ignore_index=True)


In [55]:
df

,data_week,casos_est,casos_est_min,casos_est_max,casos,proba_disse>1,incidência_100khab,disseminação,população,tempmin,umidmax,umidmed,umidmin,tempmed,tempmax,municipio,estado,longitude,latitude
0,2023-10-01,55.0,55.0,55.0,55.0,0.944330,18.022800,1.388341,305169.0,20.959643,91.241314,76.873457,56.018786,24.923086,30.577214,Limeira,SP,-47.3970,-22.56600
1,2023-10-01,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2764.0,23.018014,67.771043,37.579657,20.382600,30.257000,37.612914,Vila Nova Do Piauí,PI,-40.9345,-7.13272
2,2023-10-01,41.0,41.0,41.0,41.0,0.846745,30.862804,1.269150,132846.0,20.062329,91.504686,79.968871,62.380986,23.369157,28.510557,Valinhos,SP,-46.9974,-22.96980
3,2023-10-01,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,5920.0,22.509514,83.481329,65.289900,44.011114,27.338014,33.406600,Itambaracá,PR,-50.4097,-23.01810
4,2023-10-01,0.0,0.0,0.0,0.0,0.500000,0.000000,0.000000,5978.0,21.165000,90.037771,71.179529,45.214857,25.685800,31.962171,Feira Nova,PE,-35.3801,-7.94704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330868,2024-11-17,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2559.0,17.587400,93.958050,83.603150,71.592550,20.182183,23.185600,herveiras,RS,-52.6553,-29.45520
330869,2024-11-17,5.0,3.0,16.0,3.0,0.910051,23.431276,2.780582,21339.0,17.866150,91.664200,80.269033,63.516350,20.984583,25.042167,herval d'oeste,SC,-51.4917,-27.19030
330870,2024-11-17,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,6380.0,17.599667,86.266767,74.856517,59.026250,20.072583,23.484333,herval,RS,-53.3944,-32.02400
330871,2024-11-17,34.0,17.0,74.0,10.0,1.000000,1568.989400,7.521310,2167.0,21.949767,94.378850,79.364567,63.151500,25.530783,29.600400,guarinos,GO,-49.7006,-14.72920


In [56]:
df.to_csv('df_dengue_2023_2024.csv', index=False)

In [50]:
import pandas as pd

# Ler os arquivos CSV
df = pd.read_csv('../src/df_combined_dengue_oct_nov.csv')  # Substitua pelo nome correto do seu arquivo
df_citys = pd.read_csv('../data_sus/muni_lat_long.csv', index_col=0)

# Padronizar os nomes dos municípios em ambos os DataFrames
df['municipio'] = df['municipio'].str.strip().str.lower()
df_citys['municipio'] = df_citys['municipio'].str.strip().str.lower()

# Realizar o merge com base no nome padronizado
df_combined = pd.merge(df, df_citys[['municipio', 'estado', 'longitude', 'latitude']], on='municipio', how='left')

# Salvar o resultado em um novo arquivo CSV
df_combined.to_csv('news_data.csv', index=False)

print("Merge concluído com sucesso!")


Merge concluído com sucesso!


In [51]:
df_combined

,data_week,casos_est,casos_est_min,casos_est_max,casos,proba_disse>1,incidência_100khab,disseminação,população,tempmin,umidmax,umidmed,umidmin,tempmed,tempmax,municipio,estado,longitude,latitude
0,2024-10-06,1.0,1,1,1,0.673391,7.37844,1.752898,13553.0,23.373200,89.251443,68.761186,39.811286,26.904829,32.496857,acarape,CE,-38.7055,-4.22083
1,2024-10-13,1.0,1,1,1,0.608277,7.37844,1.394314,13553.0,23.925957,87.362200,68.116057,41.753457,27.260514,32.483443,acarape,CE,-38.7055,-4.22083
2,2024-10-20,0.0,0,0,0,0.000000,0.00000,0.000000,13553.0,23.630329,85.834357,67.937714,41.753757,26.794743,31.804929,acarape,CE,-38.7055,-4.22083
3,2024-10-27,0.0,0,0,0,0.000000,0.00000,0.000000,13553.0,23.601157,87.817057,68.694686,41.236457,27.073686,32.572586,acarape,CE,-38.7055,-4.22083
4,2024-11-03,1.0,1,1,1,0.782203,7.37844,2.821794,13553.0,23.849643,85.917271,67.393771,40.079614,27.249500,32.541771,acarape,CE,-38.7055,-4.22083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38733,2024-10-20,0.0,0,0,0,0.000000,0.00000,0.000000,2508.0,19.852943,92.669286,79.022771,63.096486,23.360429,27.718214,óleo,SP,-49.3419,-22.94350
38734,2024-10-27,0.0,0,0,0,0.000000,0.00000,0.000000,2508.0,18.063200,87.909543,72.059871,55.834843,22.403014,27.279543,óleo,SP,-49.3419,-22.94350
38735,2024-11-03,0.0,0,0,0,0.000000,0.00000,0.000000,2508.0,20.013386,96.153043,88.319943,76.657643,22.006229,24.942386,óleo,SP,-49.3419,-22.94350
38736,2024-11-10,0.0,0,0,0,0.000000,0.00000,0.000000,2508.0,17.354629,88.559871,71.591329,53.573400,22.166886,27.374714,óleo,SP,-49.3419,-22.94350


In [41]:
df_main = pd.read_csv('../data_sus/new_data/df_dengue_2023_2024.csv', index_col=0)

In [45]:
df_main

,casos_est,casos_est_min,casos_est_max,casos,proba_disse>1,incidência_100khab,disseminação,população,tempmin,umidmax,umidmed,umidmin,tempmed,tempmax,municipio,estado,longitude,latitude
data_week,,,,,,,,,,,,,,,,,,
2023-10-01,55.0,55.0,55.0,55.0,0.944330,18.022800,1.388341,305169.0,20.959643,91.241314,76.873457,56.018786,24.923086,30.577214,Limeira,SP,-47.3970,-22.56600
2023-10-01,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2764.0,23.018014,67.771043,37.579657,20.382600,30.257000,37.612914,Vila Nova Do Piauí,PI,-40.9345,-7.13272
2023-10-01,41.0,41.0,41.0,41.0,0.846745,30.862804,1.269150,132846.0,20.062329,91.504686,79.968871,62.380986,23.369157,28.510557,Valinhos,SP,-46.9974,-22.96980
2023-10-01,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,5920.0,22.509514,83.481329,65.289900,44.011114,27.338014,33.406600,Itambaracá,PR,-50.4097,-23.01810
2023-10-01,0.0,0.0,0.0,0.0,0.500000,0.000000,0.000000,5978.0,21.165000,90.037771,71.179529,45.214857,25.685800,31.962171,Feira Nova,PE,-35.3801,-7.94704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-06,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,10780.0,16.433576,74.161986,46.992743,26.117586,23.709452,30.630952,Ibiraci,MG,-47.1222,-20.46110
2024-10-06,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,73969.0,21.774469,80.153957,57.648643,34.003245,26.774583,32.697890,Canindé,CE,-39.3155,-4.35162
2024-10-06,5.0,0.0,47.0,0.0,0.433792,13.372917,0.905883,37389.0,21.448767,72.131533,51.263445,33.580871,26.520593,31.562867,Sento Sé,BA,-41.8786,-9.74138


In [ ]:
import pandas as pd

# Ler os dois arquivos CSV em DataFrames
df1 = pd.read_csv('../data_sus/new_data/df_dengue_2023_2024.csv')
df2 = pd.read_csv('../src/news_data.csv')

# Concatenar os dois DataFrames
df_concatenado = pd.concat([df1, df2], ignore_index=True)

# Salvar o resultado em um novo arquivo CSV
df_concatenado.to_csv('arquivo_concatenado.csv', index=False)

print("Arquivos concatenados com sucesso!")



In [43]:
df_concatenado

,casos_est,casos_est_min,casos_est_max,casos,proba_disse>1,incidência_100khab,disseminação,população,tempmin,umidmax,...,estado,longitude,latitude,data_week,estado_x,longitude_x,latitude_x,estado_y,longitude_y,latitude_y
0,55.0,55.0,55.0,55.0,0.944330,18.022800,1.388341,305169.0,20.959643,91.241314,...,SP,-47.3970,-22.56600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2764.0,23.018014,67.771043,...,PI,-40.9345,-7.13272,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41.0,41.0,41.0,41.0,0.846745,30.862804,1.269150,132846.0,20.062329,91.504686,...,SP,-46.9974,-22.96980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,5920.0,22.509514,83.481329,...,PR,-50.4097,-23.01810,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.500000,0.000000,0.000000,5978.0,21.165000,90.037771,...,PE,-35.3801,-7.94704,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629009,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,10780.0,16.433576,74.161986,...,NaN,NaN,NaN,2024-10-06,MG,-47.1222,-20.46110,MG,-47.1222,-20.46110
629010,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,73969.0,21.774469,80.153957,...,NaN,NaN,NaN,2024-10-06,CE,-39.3155,-4.35162,CE,-39.3155,-4.35162
629011,5.0,0.0,47.0,0.0,0.433792,13.372917,0.905883,37389.0,21.448767,72.131533,...,NaN,NaN,NaN,2024-10-06,BA,-41.8786,-9.74138,BA,-41.8786,-9.74138
629012,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,6307.0,17.010833,93.334219,...,NaN,NaN,NaN,2024-10-06,MG,-40.4296,-16.73640,MG,-40.4296,-16.73640


In [53]:
# Converter a coluna 'data_week' para o tipo datetime
df_combined['data_week'] = pd.to_datetime(df_combined['data_week'])

# Definir o intervalo de datas para o filtro
data_inicio = '2024-10-13'
data_fim = '2024-11-17'

# Aplicar o filtro
df_combined = df_combined[(df_combined['data_week'] >= data_inicio) & (df_combined['data_week'] <= data_fim)]

df_combined.sort_values(by='data_week', inplace=True)
df_combined.to_csv('df_combined_dengue_2023_2024.csv', index=False)